In [1]:
import numpy as np
import scipy as sp

# Define Hamiltonian
We want a single body Hamiltonian as a $d \times d$ matrix. We can either choose a specific Hamiltonian (TFI, AKLT, cluster state) or a random Hamiltonian.

In [2]:
# Random Hamiltonian
d = 32
H = np.random.rand(d,d) + 1.j * np.random.rand(d,d)
H = H + H.conj().T

In [3]:
# Periodic TFI
def kron(ops):
    op = 1
    for o in ops:
        op = np.kron(op, o)
    return op

Z = np.array([[1,0],[0,-1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1.j],[1.j,0]])
Id = np.eye(2)

def build_Hamiltoinan(L, Jz, hx, hz):
    # Build full 2^L x 2^L Hamiltonian

    H = np.zeros((2**L, 2**L), dtype=np.float64)

    for i in range(L):
        H += hx * kron([Id]*i + [X] + [Id]*(L-i-1))
        H += hz * kron([Id]*i + [Z] + [Id]*(L-i-1))

    Ising = [Z, Z] + [Id] * (L-2)
    for i in range(L):
        H += Jz * kron(Ising)
        # Shift the first element to the last; periodic BCs!
        Ising.append(Ising.pop(0))
    return H
H = build_Hamiltoinan(5, 1, -1.4, 0.9045)
d = H.shape[0]

In [4]:
L, U = np.linalg.eigh(H)
assert np.isclose(np.linalg.norm(U @ np.diag(L) @ U.conj().T - H), 0.0)

In [5]:
print(L)

[-8.32466182 -6.65200683 -6.65200683 -4.94094651 -4.94094651 -3.93153947
 -3.93153947 -3.89596497 -2.5123372  -2.5123372  -2.2159058  -1.60461911
 -1.60461911 -1.26609384 -1.26609384 -0.24629112  0.08191031  0.08191031
  1.18241615  1.18241615  1.39402751  2.341513    2.341513    2.55024633
  2.55024633  3.58787785  4.90650872  4.90650872  5.84494845  5.84494845
  6.49485018 11.20606816]


# Define Kraus operators for discrete quantum channel

In [6]:
L_shift = L - np.min(L)
print(L_shift)
H_shift = U @ np.diag(L_shift) @ U.conj().T

[ 0.          1.672655    1.672655    3.38371532  3.38371532  4.39312235
  4.39312235  4.42869686  5.81232462  5.81232462  6.10875603  6.72004271
  6.72004271  7.05856798  7.05856798  8.0783707   8.40657213  8.40657213
  9.50707798  9.50707798  9.71868933 10.66617482 10.66617482 10.87490815
 10.87490815 11.91253967 13.23117054 13.23117054 14.16961027 14.16961027
 14.819512   19.53072999]


In [7]:
beta = 0.1
gamma_1 = sp.linalg.expm(-beta * H_shift)
assert np.isclose(np.linalg.norm(U @ sp.linalg.expm(-beta * np.diag(L_shift)) @ U.conj().T - gamma_1), 0.0)

In [8]:
gamma_2 = U @ np.diag(np.sqrt(1 - np.exp(-2*beta*L_shift))) @ U.conj().T

In [9]:
P = np.roll(np.eye(d), 1, axis=1)
P_gamma_2 = P @ gamma_2

In [10]:
assert np.isclose(np.linalg.norm(np.eye(d) - (gamma_1.conj().T @ gamma_1 + gamma_2.conj().T @ gamma_2)), 0.0)
assert np.isclose(np.linalg.norm(np.eye(d) - (gamma_1.conj().T @ gamma_1 + P_gamma_2.conj().T @ P_gamma_2)), 0.0)

In [11]:
assert np.isclose(np.linalg.norm(gamma_1 - gamma_1.conj().T), 0.0)
assert np.isclose(np.linalg.norm(gamma_2 - gamma_2.conj().T), 0.0)
assert not np.isclose(np.linalg.norm(P_gamma_2 - P_gamma_2.conj().T), 0.0)

Both $\Gamma_1$ and $\Gamma_2$ are Hermitian, as they should be. Both are matrix exponentials of Hermitian matrices with a real exponent (imaginary time evolution and all). Additionally, we have checked that $\sum_i K_i^\dagger K_i = 1$. $P \Gamma_2$ is no longer Hermitian since we have applied a permutation matrix.

# Define vectorized channel
We want the superoperator $E = \sum_i K_i \otimes \overline{K_i}$, given Kraus operators $K_i$.

In [12]:
E = np.kron(gamma_1, gamma_1.conj()) + np.kron(gamma_2, gamma_2.conj())
print(E.shape)

(1024, 1024)


In [13]:
P_E = np.kron(gamma_1, gamma_1.conj()) + np.kron(P_gamma_2, P_gamma_2.conj())
print(P_E.shape)

(1024, 1024)


In [14]:
assert np.isclose(np.linalg.norm(E - E.conj().T), 0.0)
assert not np.isclose(np.linalg.norm(P_E - P_E.conj().T), 0.0)

The superoperator without permutation is Hermitian. This makes sense since all Kraus operators are. However, the superoperator isn't.

In [15]:
L2, U2 = np.linalg.eigh(E)
assert np.isclose(np.linalg.norm(E - U2 @ np.diag(L2) @ U2.conj().T), 0.0)
#assert np.isclose(np.linalg.norm(E - U2 @ np.diag(L2) @ sp.linalg.inv(U2)), 0.0)

Columns of $U2$ are the right eigenvectors. I am using `np.linalg.eig` rather than `np.linalg.eigh` since the fixed point eigenvectors for `eigh` aren't Hermitian when viewed as density matrices. I am not sure why this is the case.

In [16]:
print(np.max(L2))
L2_index = np.where(np.isclose(np.abs(L2), 1.0, atol=1.e-14, rtol=1.e-14))[0]
print(L2_index, len(L2_index))

1.0000000000000022
[ 968  969  970  971  972  973  974  975  976  977  978  979  980  981
  982  983  984  985  986  987  988  989  990  991  992  993  994  995
  996  997  998  999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009
 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023] 56


In [17]:
L2_sort = np.abs(L2)
L2_sort.sort()
print(L2_sort[-len(L2_index)-1:])

[0.99999553 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.        ]


Why are there multiple ($d$ for random Hamiltonian) fixed points? I guess there isn't a unique fixed point. This is because we haven't included the permutation. Need to understand why there are $d$ fixed points. I should look into purity and energies of each of the fixed points.

In [18]:
L3, U3 = np.linalg.eig(P_E)
assert np.isclose(np.linalg.norm(P_E - U3 @ np.diag(L3) @ sp.linalg.inv(U3)), 0.0, atol=1.e-7, rtol=1.e-7)

For the non-Hermitian matrices, we need to use the inverse rather than the Hermitian conjugate.

In [19]:
print(np.max(L3))
L3_index = np.where(np.isclose(np.abs(L3), 1.0, atol=1.e-14, rtol=1.e-14))[0]
print(L3_index, len(L3_index))

(1.0000000000000064+0j)
[382] 1


Now, with the permutation inserted, we have a unique fixed point (at least for our random Hamiltonian)! The permutation is doing something.

In [20]:
L3_sort = np.abs(L3)
L3_sort.sort()
print(L3_sort[-len(L3_index)-1:])

[0.99542899 1.        ]


We have a spectral gap!

# Investigate fixed points

## Permutation channel

In [21]:
L_fp = []
valid_fps = []
for i in L3_index:
    L_fp.append(L3[i])
    fp = U3[:,i].reshape(d,d)
    assert np.isclose(np.linalg.norm(fp - fp.conj().T), 0.0)
    fp = fp + fp.conj().T
    fp /= fp.trace()
    print(i, L3[i], np.linalg.eigh(fp)[0])
    valid_fps.append(fp)

382 (1.0000000000000064+0j) [1.06621347e-14 1.25511876e-14 1.45611982e-14 1.67108557e-14
 1.69214419e-14 1.84295240e-14 2.07308289e-14 2.13762744e-14
 2.15610966e-14 2.27333668e-14 2.29906162e-14 2.55920192e-14
 2.58722114e-14 2.62757026e-14 2.64591974e-14 2.71943442e-14
 2.82958184e-14 2.90688542e-14 2.99472957e-14 3.00718168e-14
 3.08281252e-14 3.29960104e-14 3.41304039e-14 3.41344198e-14
 3.48525777e-14 3.65484511e-14 3.87294108e-14 4.42884006e-14
 4.43945563e-14 5.57170406e-14 7.82792552e-14 1.00000000e+00]


In [22]:
print("GROUND STATE ENERGY:", L[0])
for i, vfp in enumerate(valid_fps):
    print("Fixed point:", i)
    print("Purity:", np.trace(vfp @ vfp))
    print("Energy:", np.trace(H @ vfp))

GROUND STATE ENERGY: -8.324661821720511
Fixed point: 0
Purity: (0.9999999999981722+0j)
Energy: (-8.324661821713466+0j)


For the permutation channel, the single fixed point is the ground state; it's a pure quantum state and has the ground state energy.

## Non-permutation channel
The eigenvectors form a basis for the fixed point subspace, but they themselves are not Hermitian or PSD.

In [23]:
L_fp = []
valid_fps = []
for i in L2_index:
    L_fp.append(L2[i])
    fp = U2[:,i].reshape(d,d)
    if np.linalg.norm(fp - fp.conj().T) < 1.e-4:
    #assert np.isclose(np.linalg.norm(fp - fp.conj().T), 0.0)
        fp = fp + fp.conj().T
        fp /= fp.trace()
        print(i, L2[i], np.linalg.eigh(fp)[0])
        valid_fps.append(fp)

988 1.0 [-1.53406542e-13 -1.22077717e-13 -1.16102354e-13 -1.06846219e-13
 -7.12706890e-14 -6.86631958e-14 -6.80776293e-14 -5.82402260e-14
 -4.68312099e-14 -2.27611619e-14 -1.09425498e-14 -8.43955205e-15
 -2.28413139e-17  2.85219664e-15  3.08623024e-15  1.72994297e-14
  2.52368980e-14  3.04780809e-14  9.31558217e-14  1.01351744e-13
  1.12556880e-13  1.18379627e-13  1.57107973e-13  1.61961127e-13
  7.35877274e-04  1.54208466e-03  3.78504799e-03  6.74366451e-03
  3.49035737e-02  5.60556057e-02  1.55871087e-01  7.40363059e-01]


In [24]:
print("GROUND STATE ENERGY:", L[0])
for i, vfp in enumerate(valid_fps):
    print("Fixed point:", i)
    print("Purity:", np.trace(vfp @ vfp))
    print("Energy:", np.trace(H @ vfp))

GROUND STATE ENERGY: -8.324661821720511
Fixed point: 0
Purity: 0.5768564683016253
Energy: 9.522499999979717
